<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#새로-띄워진-창에서-로그인을-한다" data-toc-modified-id="새로-띄워진-창에서-로그인을-한다-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>새로 띄워진 창에서 로그인을 한다</a></span></li></ul></div>

# 새로 띄워진 창에서 로그인을 한다

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import time
import requests
import re
import glob
import os
import re
import numpy as np


driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)
driver.get('https://nid.naver.com/oauth2.0/authorize?svctype=0&response_type=code&client_id=C9hwybENgOtF&state=IWK3A3NUJQVTFVU2KJPK2DRAZ6L7OXGBFYZHHTH3GMTTXKI6A6ALVCF6SFNE3OLHQ4JVWLE6HFC7Q===&redirect_url=https%3A%2F%2Fauth.band.us%2Fexternal_account_login%3Ftype%3Dnaver')


In [2]:
class_dic = {}
class_dic['PA3A'] = 'https://band.us/band/85178180'
class_dic['PB3A'] = 'https://band.us/band/85178672'
class_dic['PD3A'] = 'https://band.us/band/85186142'
class_dic['PJ3A'] = 'https://band.us/band/85186150'
class_dic['PURPLE'] = 'https://band.us/band/85186110'
class_dic['RED'] = 'https://band.us/band/85186132'
class_dic['PDEE'] = 'https://band.us/band/85237203'
class_dic['WSCN'] = 'https://band.us/band/85232892'
class_dic['PSCN'] = 'https://band.us/band/85275199'

class_ids = list(class_dic.keys())

In [3]:
month = 9
week = 5

In [7]:
a = Homework()
for class_id in class_ids:
    a.get_member_info(class_id)
a.export_member_info()

In [8]:
for class_id in class_ids[:-2]:
    tags = a.pds_tag(2) \
#          + a.voca_tag(0) \
#          + a.pareto_tag()
    test_tag = a.test_tag()
    a.export_excel(tags, test_tag)


PA3A p-9-5-1 댓글 수:  39
-------------------- 39
PA3A p-9-5-2 댓글 수:  38
-------------------- 38
PB3A p-9-5-1 댓글 수:  28
-------------------- 28
PB3A p-9-5-2 댓글 수:  28
-------------------- 28
PD3A p-9-5-1 댓글 수:  27
-------------------- 27
PD3A p-9-5-2 댓글 수:  26
-------------------- 26
PJ3A p-9-5-1 댓글 수:  24
-------------------- 24
PJ3A p-9-5-2 댓글 수:  24
-------------------- 24
PURPLE p-9-5-1 댓글 수:  45
-------------------- 45
PURPLE p-9-5-2 댓글 수:  44
-------------------- 44
RED p-9-5-1 댓글 수:  24
-------------------- 24
RED p-9-5-2 댓글 수:  24
-------------------- 24
PDEE p-9-5-1 댓글 수:  8
-------------------- 8
PDEE p-9-5-2 댓글 수:  8
-------------------- 8


In [9]:
for class_id in class_ids[-2:]:
    tags = a.special_tag(2)
    test_tag = 0
    a.export_excel(tags, test_tag)

WSCN s-9-5-1 댓글 수:  19
-------------------- 19
WSCN s-9-5-2 댓글 수:  19
-------------------- 19


In [5]:
class Homework:
     
    def __init__(self):
#         self.class_id = class_id
#         self.month = month
#         self.week = week
        
        self.member_dict = {}
        
# -----------------------------------  
   
    def get_member_info(self, class_id):
        url = class_dic[class_id]+'/member'
        driver.get(url)
        time.sleep(1.5)
        
        name_list = []
        names = driver.find_elements_by_css_selector("span.ellipsis")

        for name in names[2:]:
            name_list.append(name.text)

        self.member_dict[class_id] = pd.Series(sorted(name_list))

    def export_member_info(self):
        df = pd.DataFrame(self.member_dict) # 학생명단 
        df.to_excel(f"{week}주차_member.xlsx", index=False)
        
# ------------------------------------            
    def url(self, tag):
        self.tag = tag
        url = class_dic[class_id] + f'/hashtag/{tag}'
        driver.get(url)
        time.sleep(1)  
        
    def click_comment(self):
        try: 
            driver.find_element_by_class_name('commentToggle._commentToggleBtn').click()
            time.sleep(0.5)

            for _ in range(3):
                try:
                    driver.find_element_by_class_name('prevComment').click()
                    time.sleep(0.2)
                except :
                    pass
        except :
            pass
        
# -------------------------------
        
    def page_source(self):
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        students = soup.select('.itemWrap')
        return students
    
    def get_comment(self):
        try:
            print(class_id, self.tag, "댓글 수: ", 
                  driver.find_element_by_css_selector("span.count").text)
            student_data = {}
            students = self.page_source() # page_source 함수 실행 
            print("--------------------", len(students))
            for student in students :
                name = student.select('button > strong')[0].text
                try :
                    score = int(student.select('p')[0].text)
                except :
                    score = "O"
                student_data[name] = score 
            self.df[self.tag] = student_data
        except:
            pass
        
        
# ---------------------------------------       
    def get_test(self): 
        band_name = driver.find_element_by_class_name('uriText').text
        try : 
            # 퀴즈박스 클릭
            driver.find_element_by_class_name('item.-quiz').click() 

            scroll_area = driver.find_element_by_xpath('//*[@id="wrap"]/div[2]/div/div/section')
            action = webdriver.ActionChains(driver)
            action.move_to_element(scroll_area).perform()
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_area)
            time.sleep(1)

            # 퀴즈명 가져오기 
            quiz_name = driver.find_element_by_class_name('addTitle')
            quiz_name = quiz_name.text.replace("~", "_")

            driver.find_element_by_class_name('button._btnOpenTakerList').click() # 응답자 보기 클릭
            time.sleep(0.5)
            driver.find_element_by_class_name('btnMoreMenu._btnHeaderMoreMenu').click() # 점 3개 클릭
            time.sleep(0.5)
            driver.find_element_by_class_name('btnMenuText._btnDownloadGradingResult').click() # 채점결과 다운로드 

            time.sleep(2)

            # 저장된 파일 불러오기 
            add = f'C:/Users/hmkang/Downloads/{band_name}_채점결과_{quiz_name}_' + r'{}_{}.xlsx'
            filename = glob.glob(add.format("*", "*"))[0]
            test_df = pd.read_excel(filename, header=3) # 3번째 줄부터 불러와라 

            # 멤버 및 점수만 뽑아내기 
            # test_df['멤버'] = test_df['멤버'].str.extract(r'([가-힣].*)')
            test_df = test_df.dropna()
            test_df = test_df[['멤버', '점수']]
            test_df = test_df.rename(columns = {'멤버':'name', '점수':self.tag})
 
            return test_df 
        except :
            pass
        
        
# --------------------------------------
    def pds_tag(self, num):
        return [f'p-{month}-{week}-{i}' for i in range(1, num+1)]

    def voca_tag(self, num):
        return [f'v-{month}-{week}-{i}' for i in range(1, num+1)]

    def special_tag(self, num):
        return [f's-{month}-{week}-{i}' for i in range(1, num+1)]
    
    def test_tag(self): # 
        return [f't-{month}-{week}']
    
    def pareto_tag(self):
        return [f'1500-{month}-{week}']
    
# --------------------------------------------------    
    def comment_scrap(self, tags): 
        self.df = {}
        for tag in tags:
            self.url(tag)
            self.click_comment()
            self.get_comment()
        return pd.DataFrame(self.df).reset_index().rename(columns={'index':"name"})  


    def excel_scrap(self, test_tag): 
        for tag in test_tag:
            self.url(tag)
            return self.get_test()
        
# --------------------------------------------------    
    def export_excel(self, tags, test_tag=0):
        
        comment_df = self.comment_scrap(tags)
        
        self.member_dict[class_id].name = 'name' # 프레임으로 변경시 컬럼 이름이 됨 
        name_df = self.member_dict[class_id].to_frame() # 시리즈 형태로 저장된 밸류를 프레임으로 변환 
        name_df = name_df.dropna()
        
        df = pd.merge(name_df, comment_df, on='name', how='outer')
    
        try : 
            test_df = self.excel_scrap(test_tag) # 테스트가 있다면 ... 
            df = pd.merge(df, test_df, on='name', how='outer')
        except:
            pass 
        
        df = df.fillna("X")
        df.to_excel(f'{class_id}_{week}주차.xlsx', index=False)
        
        
        
    


In [6]:
class Export:
    
    def __init__(self, class_id, month, week, pre=0):
        self.class_id = class_id
        self.month = month
        self.week = week 
        self.pre = pre

    def week_df(self):
        df = pd.read_excel(f'./{self.week}주차/{self.class_id}_{self.week}주차.xlsx')
        try :
            week_pre = pd.read_excel(f'./{self.pre}주차/{self.pre}주차_comment.xlsx', 
                                 sheet_name=f'{self.class_id}').drop(columns=['life'])
            df = pd.merge(week_pre, df, on='name', how='outer')
        except :
            pass
        return df
        
    def heart(self):
        df = self.week_df()
        df['life'] = df.isin(["X"]).sum(axis=1)
        for j in range(self.month-1, self.month+1):
            for i in range(1, self.week+1):
                try :
                    df['life'] = np.where(df[f't-{j}-{i}'] == 15, df['life'] -1, df['life'])
                except :
                    pass
        df['life'] = df['life'].apply(lambda x : (7-x) * "🧡")
        df = df.sort_values(by=['name'])
        return df    

In [10]:
# 입력하세요 ----------
month, week = 9, 5
# -------------------
pre = week-1

with pd.ExcelWriter(f'{week}주차_comment.xlsx') as writer:  
    for class_id in class_ids:
        df = Export(class_id, month, week, pre).heart()
        df.to_excel(writer, sheet_name=f'{class_id}', index=False)